In [1]:
import tensorflow as tf
import _pickle as cPickle
import numpy as np
from random import shuffle
import matplotlib
import matplotlib.pyplot as plt

# chuyển lại default interpolation về 'bilinear' như matplotlib 1.4.0, thay vì 'nearest' như 2.0.0
# chế độ 'nearest' tự động resample, nên ta cũng phải chỉnh lại thông số này
# https://matplotlib.org/users/dflt_style_changes.html#interpolation 
matplotlib.rcParams['image.interpolation'] = 'bilinear'
matplotlib.rcParams['image.resample'] = False

In [1]:
# neuronX = number of neurons in layer X

# convolution
neurons1 = 32
neurons2 = 64
neurons3 = 128

# FC
neurons4 = 2**8
neurons5 = 2**7

# others
LEARNING_RATE = 1e-4
KEEP_PROBABILITY = 0.9

So 3 rows per image (for R G B); flatten (32x32 = 1024)

In [2]:
def unpickle(filename):
    fo = open(filename, 'rb')
    fdict = cPickle.load(fo, encoding='latin1')
    fo.close()
    return fdict
def flatten_image(image):
    return [item for sublist in image for item in sublist]

def format_data(data):
    images = []
    data_size = int(len(data)/3)
    for i in range(data_size):
        images.append(flatten_image(data[3*i:3*i+3]))
    return images

def format_label(labels):
    prep = [np.array([0]*9) for i in range(len(labels))]
    for i in range(len(labels)):
        prep[i][labels[i]] += 1
    return prep

def get_batch(filename):
    data_batch = unpickle(filename)
    data = format_data(data_batch['data'])
    labels = format_label(data_batch['labels'])
    return data, labels

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### Layer 1 (Convolutional)

In [18]:
x = tf.placeholder(tf.float32, [None, 3*32*32])
x_image = tf.reshape(x, [-1,32,32,3])
y_ = tf.placeholder(tf.float32, [None, 9])

In [20]:
W_conv1 = weight_variable([2, 2, 3, neurons1])
b_conv1 = bias_variable([neurons1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Layer 2 (Convolutional)

In [21]:
W_conv2 = weight_variable([2, 2, neurons1, neurons2])
b_conv2 = bias_variable([neurons2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Layer 3 (Convolutional)

In [22]:
W_conv3 = weight_variable([2, 2, neurons2, neurons3])
b_conv3 = bias_variable([neurons3])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

### Layer 4 (Densely connected)

In [23]:
W_fc1 = weight_variable([4 * 4 * neurons3, neurons4])
b_fc1 = bias_variable([neurons4])

h_pool3_flat = tf.reshape(h_pool3, [-1, 4*4*neurons3])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

### Layer 4-1

In [24]:
W_fc2 = weight_variable([neurons4, neurons5])
b_fc2 = bias_variable([neurons5])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

### Layer 5 (softmax, with dropout)

In [25]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [26]:
W_fc3 = weight_variable([neurons5, 9])
b_fc3 = bias_variable([9])

y_conv = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

### Training

In [27]:
sess = tf.InteractiveSession()
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [28]:
def train_cycle():
    train_order = list(range(1, 6))
    shuffle(train_order)
    for i in train_order:
        batch_xs, batch_ys = get_batch('cucumber_data/p1/data_batch_{}'.format(i))
        train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: KEEP_PROBABILITY})
    
def test_cycle():
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    test_x, test_y = get_batch('cucumber_data/p1/test_batch')
    print(sess.run(accuracy, feed_dict={x: test_x,
                                      y_: test_y,
                                      keep_prob: 1.0}))

In [29]:
for i in range(200):
    train_cycle()
    if i % 10 == 0:
        print("Round {}".format(i))
        test_cycle()

Round 0
0.131313
Round 10
0.254545
Round 20
0.268687
Round 30
0.331313
Round 40
0.369697
Round 50
0.39798
Round 60
0.432323
Round 70
0.450505
Round 80
0.464646
Round 90
0.494949
Round 100
0.513131
Round 110
0.561616
Round 120
0.541414
Round 130
0.555556
Round 140
0.577778
Round 150
0.589899
Round 160
0.606061
Round 170
0.608081
Round 180
0.612121
Round 190
0.612121


In [30]:
for i in range(200):
    train_cycle()
    if i % 10 == 0:
        print("Round {}".format(200 + i))
        test_cycle()

Round 200
0.614141
Round 210
0.640404
Round 220
0.642424
Round 230
0.654545
Round 240
0.662626
Round 250
0.674747
Round 260
0.650505
Round 270
0.666667
Round 280
0.662626
Round 290
0.664646
Round 300
0.680808
Round 310
0.678788
Round 320
0.680808
Round 330
0.686869
Round 340
0.688889
Round 350
0.692929
Round 360
0.674747
Round 370
0.688889
Round 380
0.707071
Round 390
0.682828


In [31]:
for i in range(200):
    train_cycle()
    if i % 10 == 0:
        print("Round {}".format(400 + i))
        test_cycle()

Round 400
0.70101
Round 410
0.705051
Round 420
0.692929
Round 430
0.69495
Round 440
0.70303
Round 450
0.690909
Round 460
0.69495
Round 470
0.69899
Round 480
0.69899
Round 490
0.69697
Round 500
0.723232
Round 510
0.690909
Round 520
0.69697
Round 530
0.715151
Round 540
0.715151
Round 550
0.70101
Round 560
0.717172
Round 570
0.70303
Round 580
0.717172
Round 590
0.711111
